In [ ]:
import ipywidgets as widgets
from IPython.display import display

from ipyniivue import NiiVue

In [ ]:
nv = NiiVue()
display(nv)

In [ ]:
# Document URLs to load
document_urls = [
    "https://niivue.com/demos/images/niivue.basic.nvd",
    "https://niivue.com/demos/images/niivue.drawing.nvd",
    "https://niivue.com/demos/images/niivue.mesh.nvd",
]

# Load the first document by default
nv.load_document(document_urls[0])

In [ ]:
# Create preview buttons for documents
def create_document_button(url, index):
    """Create a button for loading a document."""
    name = url.split("/")[-1]
    button = widgets.Button(
        description=f"Load {name}", layout=widgets.Layout(width="auto", margin="2px")
    )

    def on_click(b):
        nv.load_document(url)
        status_output.value = f"Loaded: {name}"

    button.on_click(on_click)
    return button


# Create buttons for each document
document_buttons = [
    create_document_button(url, i) for i, url in enumerate(document_urls)
]

# Status display
status_output = widgets.HTML(value="Loaded: niivue.basic.nvd")

In [ ]:
# Track hippocampus state
hippo_state = {"is_showing": False}
hippo_url = "https://niivue.com/demos/images/hippo.nii.gz"


def toggle_hippocampus(b):
    """Toggle the hippocampus overlay."""
    if hippo_state["is_showing"]:
        nv.volumes = nv.volumes[:-1]
        b.description = "Add Hippocampus"
        hippo_state["is_showing"] = False
        status_output.value = "Removed hippocampus overlay"
    else:
        # Add hippocampus
        nv.add_volume({"url": hippo_url, "colormap": "bluegrn", "opacity": 0.7})
        b.description = "Remove Hippocampus"
        hippo_state["is_showing"] = True
        status_output.value = "Added hippocampus overlay"


# Create toggle button
hippo_button = widgets.Button(description="Add Hippocampus", button_style="info")
hippo_button.on_click(toggle_hippocampus)

In [ ]:
def save_document(b):
    """Save the current scene as a document."""
    nv.save_document("my_document.nvd")
    status_output.value = "Document saved as 'my_document.nvd'"


# Create save button
save_button = widgets.Button(description="Save Document", button_style="success")
save_button.on_click(save_document)

In [ ]:
# Create organized layout
file_controls = widgets.VBox(
    [
        widgets.HTML("<h3>File Operations</h3>"),
        widgets.HBox([save_button]),
        widgets.HTML("<h3>Load Documents</h3>"),
        widgets.VBox(document_buttons),
        widgets.HTML("<h3>Volume Controls</h3>"),
        widgets.HBox([hippo_button]),
        widgets.HTML("<h3>Status</h3>"),
        status_output,
    ]
)

# Display controls
display(file_controls)
display(nv)